In [64]:
import glob
import ast
import gc
import os
import re
from datetime import datetime

import matplotlib.pyplot as plt
import nltk
import numpy as np
import pandas as pd
import seaborn as sns
from IPython.display import SVG
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.layers import (LSTM, Activation, Conv1D, Dense, Dropout, Embedding,
                          Flatten, Input, MaxPooling1D, concatenate, dot)
from keras.models import Model, Sequential
from keras.utils.vis_utils import model_to_dot
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from scipy.spatial.distance import correlation, cosine
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import pairwise_distances
from sklearn.metrics.pairwise import cosine_similarity, linear_kernel
from sklearn.model_selection import train_test_split
from surprise import (NMF, SVD, BaselineOnly, Dataset, KNNBasic,
                      NormalPredictor, Reader)
from surprise.model_selection import (GridSearchCV, KFold, RandomizedSearchCV,
                                      cross_validate)
from tqdm import tqdm

tqdm.pandas()

In [65]:
df_vids = pd.concat([pd.read_pickle(i[2:]) for i in glob.glob(
    './pickles/individual/*full.pkl')], ignore_index=True)  # * local
df_vids.dropna(inplace=True)
# minimum number of videos before a game is considered
df_vids = df_vids.groupby("game").filter(lambda x: len(x) >= 5)
df_vids.sample(10)

,channel,title,videoID,date,likes,comments,views,game
44220,PartyElite,Total War: Warhammer 2 (Vortex) - HURRIKHAINE ...,mS2ObIM5dxY,2018-06-04T05:00:04Z,126.0,29.0,"4,177.0",Total War: WARHAMMER II
62891,theRadBrad,The Amazing Spider-Man - Gameplay Walkthrough ...,4OSSslZbvbA,2012-07-07T20:30:08Z,"3,829.0",373.0,"434,519.0",Spider-Man: Shattered Dimensions
53300,quill18,Let's Play Democracy 3: Clones and Drones,oI0gcZGOtA4,2014-12-02T02:00:03Z,"1,020.0",95.0,"58,109.0",Democracy 3
58662,theRadBrad,FALLOUT 76 Walkthrough Gameplay Part 2 - COLLI...,cGDQIGN0YjE,2018-11-18T00:30:02Z,"13,123.0",884.0,"460,602.0",Fallout 76
9904,DanTDM,Minecraft | TRAYAURUS' CASTLE!! | Pixel Painte...,cI3omQBvNMk,2015-09-18T18:32:59Z,"26,043.0","21,000.0","2,843,967.0",Minecraft
51252,quill18,Let's Suck at Hearthstone: Renounce Warlock [W...,7c1GWShdrlk,2016-04-27T20:22:07Z,258.0,35.0,"10,545.0",Hearthstone
6902,Blitz,Cities Skylines - Ep. 44 - The Final Walkthrou...,LbmFCRRc-JM,2015-04-13T19:00:00Z,594.0,56.0,"49,609.0",Cities: Skylines
17781,jacksepticeye,PREPARE FOR MONKEE BUTTS | Plague Inc. Evolved...,58B1uiPe72w,2015-10-22T16:00:00Z,"68,772.0","6,000.0","3,337,794.0",Plague Inc: Evolved
61649,theRadBrad,Outlast Gameplay Walkthrough Part 6 - Run Hide...,GpZIhEOYI78,2013-09-11T03:51:22Z,"15,652.0","2,500.0","695,892.0",Outlast
8013,DanTDM,Dropping My NETHERITE ARMOR in LAVA! (Minecraf...,Hu6i0_oRU_U,2020-05-12T18:06:35Z,"174,151.0","14,000.0","5,715,791.0",Minecraft


In [66]:
df_vids.columns

Index(['channel', 'title', 'videoID', 'date', 'likes', 'comments', 'views',
       'game'],
      dtype='object')

In [67]:
df_vids = df_vids[['channel', 'title', 'date', 'likes', 'comments', 'views',
       'game']].copy()
df_vids.rename(columns={'channel':'Channel Title', 'title':'Video Title', 'date': 'Upload Date', 'likes': 'Likes', 'comments': 'Comments', 'views':'Views',
       'game':'Game Played'}, inplace=True)
df_vids.head()

,Channel Title,Video Title,Upload Date,Likes,Comments,Views,Game Played
0,AngryJoeShow,Is Warhammer: Man O' War Corsair BETTER than S...,2023-01-20T17:08:39Z,"1,889.0",380.0,"60,972.0",2023
1,AngryJoeShow,"AJS News - Callisto Protocol Controversy, Blac...",2023-01-17T22:58:20Z,"6,127.0",602.0,"144,970.0",The Callisto Protocol
2,AngryJoeShow,"AJS News - Wizards D&D OGL Controversy, Dead S...",2023-01-17T22:58:18Z,"6,763.0",646.0,"135,178.0",Dead Space
4,AngryJoeShow,BattleBit Remastered is BETTER than Battlefiel...,2023-01-14T23:42:19Z,"5,327.0","1,200.0","131,807.0",Battlefield 2042
8,AngryJoeShow,The Game Awards 2022 - FULL LENGTH ANGRY REACT...,2023-01-03T00:01:18Z,"2,619.0",867.0,"97,435.0",League of Legends


In [68]:
df_vids = df_vids[~df_vids['Game Played'].isin(['1996', '2000', '2004', '2009', '2010', '2011', '2012', '2013',
                                        '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023', 'VRChat', 'Youtubers Life'])].copy()
df_vids.sample(20)

,Channel Title,Video Title,Upload Date,Likes,Comments,Views,Game Played
24867,KatherineOfSky,Factorio 0.14 Ribbon World - Ep 50 - New Oil! ...,2016-11-07T07:47:04Z,42.0,9.0,"1,231.0",Factorio
51598,quill18,Let's Play Tharsis with Briarstone...and beer!,2016-01-13T23:00:02Z,810.0,62.0,"26,063.0",Tharsis
13013,H2ODelirious,HARD WORK & WIFE LEAVING ME! - RED DEAD REDEMP...,2018-12-13T21:18:20Z,"12,676.0","1,000.0","462,286.0",Red Dead Redemption 2
44404,PartyElite,Total War: Warhammer 2 (Mortal Empires) - BRAN...,2017-12-27T10:00:00Z,146.0,20.0,"3,643.0",Total War: WARHAMMER II
8040,DanTDM,Piggy is INFECTED NOW?! (Roblox Piggy),2020-03-31T16:48:13Z,"92,480.0","9,600.0","2,948,778.0",Roblox
74071,Splattercatgaming,SOMA Gameplay / Let's Play - I AM THE OMNITOOL...,2015-09-21T15:00:00Z,691.0,61.0,"25,379.0",Soma
76540,Splattercatgaming,S3 Episode 4 - Let's Play Don't Starve - Thund...,2013-03-28T12:00:28Z,54.0,13.0,"1,526.0",Don't Starve
75247,Splattercatgaming,Let's Play Interstellaria - Episode 3 - Starmap!,2014-12-01T20:00:12Z,261.0,29.0,"15,341.0",Interstellaria
60875,theRadBrad,Destiny Gameplay Walkthrough Part 8 - Sword of...,2014-09-11T20:46:05Z,"10,910.0",795.0,"459,175.0",Destiny
50531,quill18,Let's Play: Path of Exile - Summoner Witch - P...,2016-11-06T20:00:01Z,314.0,37.0,"23,013.0",Path of Exile


In [69]:
df_vids = df_vids.astype({'Likes':'int', 'Comments':'int', 'Views':'int'})
df_vids['Upload Date'] = pd.to_datetime(df_vids['Upload Date'], errors='coerce')

In [70]:
df_vids.sample(120)

,Channel Title,Video Title,Upload Date,Likes,Comments,Views,Game Played
40093,Many A True Nerd,Into the Breach - Future Perfect,2018-03-07 19:59:44+00:00,726,52,35980,Into the Breach
22276,KatherineOfSky,Whistle Stop Factorio Ep 28: MODULE TRADEOFF -...,2018-08-24 06:00:02+00:00,198,38,6010,Factorio
65676,SeaNanners Gaming Channel,TRAITOR TIME! (Trouble in Terrorist Town),2013-03-27 16:00:40+00:00,20625,958,1079214,Trouble in Terrorist Town
52491,quill18,SUPER ULM - Europa Universalis 4 Let's Play - ...,2015-06-03 22:00:02+00:00,416,60,12426,Europa Universalis IV
45952,quill18,Stellaris: Have You Been Chipped? - Ep 33,2021-05-23 13:42:29+00:00,377,14,6566,Stellaris
...,...,...,...,...,...,...,...
60094,theRadBrad,Undead Dragons in Dark Souls 3 Walkthrough Gam...,2016-03-31 21:00:00+00:00,14738,1300,681862,Dark Souls III
70884,Splattercatgaming,"Anno 1800 - Building a City, Rescuing a People!",2019-04-21 15:00:00+00:00,1574,93,22479,Anno 1800
12552,H2ODelirious,EVERYTHING TRIES TO EAT MEEEEE! | Little Nig...,2020-05-16 23:07:20+00:00,39930,3900,885670,Little Nightmares
51587,quill18,Aurora 4x: Truly Epic Space Strategy - Ep. 24,2016-01-19 17:00:03+00:00,600,97,21688,Aurora 4x


In [71]:
df_games = pd.read_pickle('./pickles/df_games.pkl')
df_games.sample(4)

,name,count,appid,requiredAge,metacritic,price,windows,mac,linux,releaseDate,acheiveCount,recs,publishers,developers,desc,header_image,genres,categories
446,Road Redemption,18,"300,380.0",0,NaN,"1,999.0",True,True,True,"Oct 4, 2017",55.0,"8,404.0",[Tripwire Presents],"[EQ-Games , Pixel Dash Studios]","<img src=""https://cdn.akamai.steamstatic.com/s...",https://cdn.akamai.steamstatic.com/steam/apps/...,"[Action, Adventure, Indie, Racing, Simulation]","[Single-player, Multi-player, Online PvP, Co-o..."
232,Halo 4,43,"1,695,792.0",18,NaN,NaN,True,False,False,,NaN,NaN,[],"[343 Industries, Splash Damage, Ruffian Games]",,https://cdn.akamai.steamstatic.com/steam/apps/...,[Action],"[Single-player, Multi-player, PvP, Online PvP,..."
162,Craft The World,60,"248,390.0",0,NaN,"1,899.0",True,True,False,"Nov 24, 2014",81.0,"11,903.0",[Black Maple Games],[Dekovir Entertainment],Craft The World is a unique sandbox strategy g...,https://cdn.akamai.steamstatic.com/steam/apps/...,"[Indie, RPG, Simulation, Strategy]","[Single-player, Multi-player, Co-op, Online Co..."
241,Mad Max,39,"234,140.0",18,73.0,"1,999.0",True,False,False,"1 Sep, 2015",49.0,"42,283.0","[Warner Bros. Games, Warner Bros. Interactive ...",[Avalanche Studios],Online functionality will be retired on Octobe...,https://cdn.akamai.steamstatic.com/steam/apps/...,"[Action, Adventure]","[Single-player, Steam Achievements, Full contr..."


In [72]:
df_games.columns

Index(['name', 'count', 'appid', 'requiredAge', 'metacritic', 'price',
       'windows', 'mac', 'linux', 'releaseDate', 'acheiveCount', 'recs',
       'publishers', 'developers', 'desc', 'header_image', 'genres',
       'categories'],
      dtype='object')

In [73]:
df_games = df_games[['name', 'count', 'metacritic', 'price',
       'releaseDate', 'desc', 'header_image', 'genres',
       'categories']].copy()
df_games.rename(columns={'name':'Game Title', 'count':'Video Count', 'metacritic':'Metacritic Score', 'price':'Price (USD)',
       'releaseDate':'Release date (steam)', 'desc':'Description', 'genres':'Genres',
       'categories':'Tags'}, inplace=True)
df_games.sample(10)

,Game Title,Video Count,Metacritic Score,Price (USD),Release date (steam),Description,header_image,Genres,Tags
312,Unturned,27,NaN,NaN,"Jul 7, 2017","<img src=""https://cdn.akamai.steamstatic.com/s...",https://cdn.akamai.steamstatic.com/steam/apps/...,"[Action, Adventure, Casual, Free to Play, Indie]","[Single-player, Multi-player, PvP, Online PvP,..."
459,Darkest Dungeon II,17,NaN,NaN,Q2 2023,Darkest Dungeon II is a roguelike road trip of...,https://cdn.akamai.steamstatic.com/steam/apps/...,"[Indie, RPG, Strategy]","[Single-player, Steam Achievements, Steam Work..."
554,Stick It to the Stickman,12,NaN,NaN,Coming soon,"<a href=""https://steamcommunity.com/linkfilter...",https://cdn.akamai.steamstatic.com/steam/apps/...,"[Action, Indie]","[Single-player, Full controller support, Remot..."
371,Counter-Strike: Global Offensive,22,83.0,NaN,21 авг. 2012,Counter-Strike: Global Offensive (CS:GO) расши...,https://cdn.akamai.steamstatic.com/steam/apps/...,"[Экшены, Бесплатно]","[Для нескольких игроков, Достижения Steam, Кон..."
380,Battlefleet Gothic: Armada,22,77.0,"1,999.0","Jan 24, 2019",<strong>Battlefleet Gothic: Armada 2</strong> ...,https://cdn.akamai.steamstatic.com/steam/apps/...,[Strategy],"[Single-player, Multi-player, PvP, Online PvP,..."
117,Outlast,79,80.0,"1,999.0","Sep 4, 2013",Hell is an experiment you can't survive in Out...,https://cdn.akamai.steamstatic.com/steam/apps/...,"[Action, Adventure, Indie]","[Single-player, Steam Achievements, Full contr..."
56,The Last of Us,132,NaN,"6,999.0","3 Mar, 2023","<img src=""https://cdn.akamai.steamstatic.com/s...",https://cdn.akamai.steamstatic.com/steam/apps/...,"[Action, Adventure]","[Single-player, Full controller support]"
405,Metro: Redux,20,NaN,"1,999.0","Aug 25, 2014",In 2013 the world was devastated by an apocaly...,https://cdn.akamai.steamstatic.com/steam/apps/...,[Action],"[Single-player, Steam Achievements, Full contr..."
39,Civilization IV,190,83.0,"1,999.0","Sep 23, 2008",Sid Meier's Civilization IV: Colonization is t...,https://cdn.akamai.steamstatic.com/steam/apps/...,[Strategy],"[Single-player, Multi-player]"
128,Don't Starve: Shipwrecked,74,77.0,"1,000.0","1 Dec, 2015",Klei Entertainment has partnered with our frie...,https://cdn.akamai.steamstatic.com/steam/apps/...,"[Adventure, Indie, Simulation]","[Single-player, Downloadable Content, Full con..."


In [74]:
df_games['Price (USD)'] = df_games['Price (USD)']/100
df_games['Release date (steam)'] = pd.to_datetime(df_games['Release date (steam)'], errors='coerce')
df_games.sample(10)

,Game Title,Video Count,Metacritic Score,Price (USD),Release date (steam),Description,header_image,Genres,Tags
165,Portal 2,59,95.0,"11,000.0",2011-04-18,Portal 2 draws from the award-winning formula ...,https://cdn.akamai.steamstatic.com/steam/apps/...,"[Action, Adventure]","[Single-player, Multi-player, Co-op, Online Co..."
328,The Stanley Parable,25,88.0,14.99,2013-10-17,The Stanley Parable is a first person explorat...,https://cdn.akamai.steamstatic.com/steam/apps/...,"[Adventure, Indie]","[Single-player, Steam Achievements, Full contr..."
580,Tasty Planet,11,NaN,5.99,2017-11-02,Eat Everything in the World!<br><br>This is th...,https://cdn.akamai.steamstatic.com/steam/apps/...,"[Action, Adventure, Casual, Indie]","[Single-player, Steam Achievements, Partial Co..."
450,The Flame in the Flood,17,73.0,14.99,2016-02-24,A rogue-like river journey through the backwat...,https://cdn.akamai.steamstatic.com/steam/apps/...,"[Action, Adventure, Indie]","[Single-player, Steam Achievements, Full contr..."
148,2018,64,NaN,19.99,2017-07-28,"<a href=""https://store.steampowered.com/app/15...",https://cdn.akamai.steamstatic.com/steam/apps/...,"[Racing, Simulation]","[Single-player, Steam Achievements, Steam Trad..."
380,Battlefleet Gothic: Armada,22,77.0,19.99,2019-01-24,<strong>Battlefleet Gothic: Armada 2</strong> ...,https://cdn.akamai.steamstatic.com/steam/apps/...,[Strategy],"[Single-player, Multi-player, PvP, Online PvP,..."
215,Getting Over It with Bennett Foddy,48,NaN,7.99,2017-12-06,<i>A game I made<br>For a certain kind of pers...,https://cdn.akamai.steamstatic.com/steam/apps/...,[Action],"[Single-player, Partial Controller Support, Co..."
207,Dead Cells,50,89.0,24.99,2018-08-06,"<img src=""https://cdn.akamai.steamstatic.com/s...",https://cdn.akamai.steamstatic.com/steam/apps/...,"[Action, Adventure, Indie]","[Single-player, Steam Achievements, Full contr..."
494,A Plague Tale: Requiem,14,NaN,49.99,2022-10-17,"<img src=""https://cdn.akamai.steamstatic.com/s...",https://cdn.akamai.steamstatic.com/steam/apps/...,"[Action, Adventure]","[Single-player, Steam Achievements, Full contr..."
504,Mirror's Edge Catalyst,14,74.0,19.99,2020-06-04,"Follow Faith, a daring free runner, as she fig...",https://cdn.akamai.steamstatic.com/steam/apps/...,"[Action, Adventure]","[Single-player, Steam Achievements, Full contr..."


In [75]:
df_games['Price (USD)'][df_games['Price (USD)'] >= 120] = np.nan

C:\Users\crystallikelaw\AppData\Local\Temp\ipykernel_18124\2599361287.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_games['Price (USD)'][df_games['Price (USD)'] >= 120] = np.nan


In [76]:
df_games.sample(10)

,Game Title,Video Count,Metacritic Score,Price (USD),Release date (steam),Description,header_image,Genres,Tags
254,Hades,32,93.0,28.99,2020-09-17,"<img src=""https://cdn.akamai.steamstatic.com/s...",https://cdn.akamai.steamstatic.com/steam/apps/...,"[Action, Indie, RPG]","[Single-player, Full controller support, Steam..."
74,The Hidden,111,NaN,99.99,2022-04-15,The Hidden Ghost is a casual-survival game whe...,https://cdn.akamai.steamstatic.com/steam/apps/...,"[Adventure, Casual, Indie, Simulation]",[Single-player]
494,A Plague Tale: Requiem,14,NaN,49.99,2022-10-17,"<img src=""https://cdn.akamai.steamstatic.com/s...",https://cdn.akamai.steamstatic.com/steam/apps/...,"[Action, Adventure]","[Single-player, Steam Achievements, Full contr..."
92,This War of Mine,97,83.0,19.99,2014-11-14,"<h2 class=""bb_tag"">Recently updated with This ...",https://cdn.akamai.steamstatic.com/steam/apps/...,"[Adventure, Indie, Simulation]","[Single-player, Steam Achievements, Full contr..."
380,Battlefleet Gothic: Armada,22,77.0,19.99,2019-01-24,<strong>Battlefleet Gothic: Armada 2</strong> ...,https://cdn.akamai.steamstatic.com/steam/apps/...,[Strategy],"[Single-player, Multi-player, PvP, Online PvP,..."
583,The Witch's House,11,NaN,14.99,2018-10-30,"<img src=""https://cdn.akamai.steamstatic.com/s...",https://cdn.akamai.steamstatic.com/steam/apps/...,"[Adventure, Casual, Indie, RPG]","[Single-player, Full controller support]"
264,Duke Nukem Forever,31,54.0,19.99,2011-06-13,"The King is Back!<br><br>\t\t\t\t\t\tCocked, l...",https://cdn.akamai.steamstatic.com/steam/apps/...,[Action],"[Single-player, Multi-player, Steam Achievemen..."
620,Call of Duty: Warzone,10,NaN,NaN,2022-11-16,"Welcome to Call of Duty®: Warzone™ 2.0, the ma...",https://cdn.akamai.steamstatic.com/steam/apps/...,"[Action, Free to Play]","[Single-player, Multi-player, PvP, Online PvP,..."
391,Super Auto Pets,21,NaN,NaN,2021-09-24,Build a team of cute pets with unique abilitie...,https://cdn.akamai.steamstatic.com/steam/apps/...,"[Free to Play, Strategy]","[Multi-player, PvP, Online PvP, In-App Purchases]"
420,From Dust,19,76.0,14.99,2011-08-17,<strong>From Dust</strong> is the latest origi...,https://cdn.akamai.steamstatic.com/steam/apps/...,"[Adventure, Simulation]",[Single-player]


In [77]:
df_games.describe()

,Video Count,Metacritic Score,Price (USD)
count,625.0,374.0,542.0
mean,74.6224,79.43850267379679,26.33559040590406
std,199.25423814943767,8.28676478936476,16.955088525722946
min,10.0,51.0,1.99
25%,17.0,74.0,14.99
50%,27.0,80.0,19.99
75%,61.0,86.0,29.99
max,"2,926.0",97.0,99.99


In [78]:
# Cleaning functions
def cleanHtml(text: str) -> str:
    """
    Removes HTML symbols from text

    Args:
        text (str): input text

    Returns:
        str: text without things like %2E in it
    """
    html = re.compile(r"<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});")
    return re.sub(html, r' ', text)


def cleanUrl(text: str) -> str:
    """
    Removes http(s) urls from text

    Args:
        text (str): input text

    Returns:
        str: output text without urls like https://google.com/
    """
    return re.sub(r"https?://\S+|www\.\S+", r' ', text)


def cleanAscii(text: str) -> str:
    """
    Remove non ASCII characters

    Args:
        text (str): input text

    Returns:
        str: text without things like carriage returns, tabs, null bytes
             removes from \x00 (null byte) to \x7f (delete)
    """
    return re.sub(r'[^\x00-\x7f]', r' ', text)


def cleanSpecChar(text: str) -> str:
    """
    Removes misc special unicode chars

    Args:
        text (str): input text

    Returns:
        str: text without emojis, pictographs, maps, flags.. 
    """
    specPattern = re.compile(
        '['
        u'\U0001F600-\U0001F64F'  # emoticons
        u'\U0001F300-\U0001F5FF'  # symbols & pictographs
        u'\U0001F680-\U0001F6FF'  # transport & map symbols
        u'\U0001F1E0-\U0001F1FF'  # flags (iOS)
        u'\U00002702-\U000027B0'
        u'\U000024C2-\U0001F251'
        ']+',
        flags=re.UNICODE)
    return specPattern.sub(r' ', text)


def cleanSpaces(text: str) -> str:
    """
    Converts multiple spaces in text to a single space

    Args:
        text (str): input text

    Returns:
        str: text with at most contiguous space
    """
    return re.sub(r'\s{2,}', r' ', text)


def fullClean(text: str) -> str:
    """
    Compiles the above cleaning functions

    Args:
        text (str): input text

    Returns:
        str: cleaned output text, lowercase
    """
    return cleanSpaces(cleanSpecChar(cleanAscii(cleanHtml(cleanUrl(text))))).strip()

df_games['Description'] = df_games['Description'].progress_apply(fullClean)

100%|██████████| 625/625 [00:00<00:00, 7181.56it/s]


In [79]:
df_games.sample(10)

,Game Title,Video Count,Metacritic Score,Price (USD),Release date (steam),Description,header_image,Genres,Tags
300,Warlock: Master of the Arcane,28,71.0,24.99,2012-05-08,The Warlock: Master of the Arcane Complete Col...,https://cdn.akamai.steamstatic.com/steam/apps/...,[Strategy],"[Single-player, Multi-player, Steam Trading Ca..."
377,Gears 5,22,82.0,39.99,2019-09-09,"From one of gaming s most acclaimed sagas, Gea...",https://cdn.akamai.steamstatic.com/steam/apps/...,"[Action, Adventure]","[Single-player, Multi-player, PvP, Online PvP,..."
483,Chivalry 2,14,82.0,39.99,2022-06-12,Return to the ultimate medieval battlefield! C...,https://cdn.akamai.steamstatic.com/steam/apps/...,[Action],"[Multi-player, PvP, Online PvP, Cross-Platform..."
275,Railroad Tycoon 3,30,80.0,9.99,2007-05-04,Railroad Tycoon 3 features 25 scenarios challe...,https://cdn.akamai.steamstatic.com/steam/apps/...,[Strategy],"[Single-player, Multi-player]"
264,Duke Nukem Forever,31,54.0,19.99,2011-06-13,"The King is Back! Cocked, loaded and ready for...",https://cdn.akamai.steamstatic.com/steam/apps/...,[Action],"[Single-player, Multi-player, Steam Achievemen..."
199,Turmoil,52,73.0,9.99,2016-06-02,"Turmoil offers players a visually charming, to...",https://cdn.akamai.steamstatic.com/steam/apps/...,"[Indie, Simulation, Strategy]","[Single-player, Multi-player, PvP, Online PvP,..."
164,The Elder Scrolls IV: Oblivion,59,94.0,14.99,2009-06-16,The Elder Scrolls IV: Oblivion Game of the Yea...,https://cdn.akamai.steamstatic.com/steam/apps/...,[RPG],"[Single-player, Steam Cloud]"
569,Call of Duty,11,NaN,89.99,2022-10-27,Existing Modern Warfare II Digital Standard Ed...,https://cdn.akamai.steamstatic.com/steam/apps/...,[Action],"[Single-player, Multi-player, PvP, Online PvP,..."
400,Call of Duty: World at War,20,83.0,19.99,2008-11-18,"Call of Duty is back, redefining war like you'...",https://cdn.akamai.steamstatic.com/steam/apps/...,[Action],"[Single-player, Multi-player, Co-op]"
144,DOOM Eternal,65,88.0,39.99,2020-03-19,Hell s armies have invaded Earth. Become the S...,https://cdn.akamai.steamstatic.com/steam/apps/...,[Action],"[Single-player, Multi-player, PvP, Online PvP,..."


In [80]:
df_vids.sample(10)

,Channel Title,Video Title,Upload Date,Likes,Comments,Views,Game Played
9521,DanTDM,Minecraft | THE MANTIS MACHINE?! | Crazy Craft...,2016-04-18 19:47:37+00:00,46723,9100,3441943,Minecraft
10307,DanTDM,Minecraft | THE CREEPER SHARD | Diamond Dimens...,2015-02-01 20:23:36+00:00,20304,4400,1728599,Minecraft
9544,DanTDM,BANK OWNER MILLIONAIRES?! | Roblox,2016-04-06 18:09:48+00:00,66129,17000,9380348,Roblox
32379,Mathas,SING ME A SONG | Darkest Dungeon The Crimson C...,2017-07-10 19:53:58+00:00,116,10,2487,Darkest Dungeon: The Crimson Court
30014,Markiplier,Drunk Minecraft #11 | PEN1S CANNON REDUX,2012-09-09 14:00:11+00:00,13403,3400,1109442,Minecraft
53929,quill18,Let's Drunk: Interplanetary - Part 2,2014-07-01 02:30:00+00:00,1516,200,62853,Kerbal Space Program
43798,PartyElite,WARHAMMER CHAOSBANE | DIABLO MEETS WARHAMMER? ...,2019-03-20 14:00:03+00:00,174,30,5438,Warhammer: Chaosbane
77014,VanossGaming,Golf It Funny Moments - Saying the Magic Word!!,2021-06-07 02:12:41+00:00,60710,723,1578296,Golf It!
19678,jacksepticeye,DMC: Devil May Cry PC - Escaping the House - G...,2013-01-28 07:16:48+00:00,698,41,23293,DmC: Devil May Cry
48489,quill18,Let's Play Cities: Skylines - Parklife - Part 6,2018-06-26 07:00:03+00:00,297,5,12204,Cities: Skylines


In [81]:
df_full = df_vids.merge(df_games, left_on="Game Played", right_on="Game Title")
df_full.sample(5)

,Channel Title,Video Title,Upload Date,Likes,Comments,Views,Game Played,Game Title,Video Count,Metacritic Score,Price (USD),Release date (steam),Description,header_image,Genres,Tags
19151,jacksepticeye,I WON in a MASSIVE Minecraft Tournament,2022-06-23 16:00:06+00:00,104296,3600,1530848,Minecraft,Minecraft,2926,NaN,19.99,2021-09-22,Fight your way through an exciting action-adve...,https://cdn.akamai.steamstatic.com/steam/apps/...,"[Action, Adventure, RPG]","[Single-player, Multi-player, Co-op, Online Co..."
35519,quill18,Let's Play Crusader Kings III - Count of Anjou...,2020-10-28 07:00:01+00:00,293,12,9598,Crusader Kings III,Crusader Kings III,337,91.0,59.99,2020-09-01,Your legacy awaits. Choose your noble house an...,https://cdn.akamai.steamstatic.com/steam/apps/...,"[RPG, Simulation, Strategy]","[Single-player, Multi-player, PvP, Online PvP,..."
3937,Many A True Nerd,The Last of Us - At Long Last,2020-06-01 21:01:33+00:00,4561,749,116980,The Last of Us,The Last of Us,132,NaN,69.99,2023-03-03,Experience the emotional storytelling and unfo...,https://cdn.akamai.steamstatic.com/steam/apps/...,"[Action, Adventure]","[Single-player, Full controller support]"
34876,theRadBrad,The Evil Within Walkthrough Gameplay Part 25 -...,2014-11-02 20:00:05+00:00,13765,1100,671860,The Evil Within,The Evil Within,89,68.0,19.99,2014-10-13,Developed by Shinji Mikami -- creator of the s...,https://cdn.akamai.steamstatic.com/steam/apps/...,[Action],"[Single-player, Steam Achievements, Full contr..."
10987,quill18,Stellaris: MegaCorp -- Mom's Friendly Robot Co...,2018-12-28 18:45:37+00:00,371,23,12611,Stellaris,Stellaris,704,78.0,34.99,2016-05-09,"Get ready to explore, discover and interact wi...",https://cdn.akamai.steamstatic.com/steam/apps/...,"[Simulation, Strategy]","[Single-player, Multi-player, Cross-Platform M..."


In [82]:
df_full.drop(columns=['Game Title'], inplace=True)
df_full.rename(columns={'Video Count': 'Game Video Count'})
df_full.sample(5)

,Channel Title,Video Title,Upload Date,Likes,Comments,Views,Game Played,Video Count,Metacritic Score,Price (USD),Release date (steam),Description,header_image,Genres,Tags
33949,Mathas,The Binding of Isaac: Afterbirth - Episode 318...,2016-09-26 20:04:48+00:00,202,23,5383,The Binding of Isaac: Rebirth,862,NaN,14.99,2014-11-04,When Isaac s mother starts hearing the voice o...,https://cdn.akamai.steamstatic.com/steam/apps/...,[Action],"[Single-player, Multi-player, Shared/Split Scr..."
6083,VanossGaming,Phasmophobia Funny Moments - Brian Michael Jac...,2022-12-07 02:08:07+00:00,67248,1300,1071257,Phasmophobia,63,NaN,13.99,2020-09-18,"Phasmophobia is a 4-player, online co-op, psyc...",https://cdn.akamai.steamstatic.com/steam/apps/...,"[Action, Indie, Early Access]","[Single-player, Multi-player, Co-op, Online Co..."
28189,Blitz,Starbound - Ep. 6 - Steel Spoons and Armor! - ...,2015-02-09 20:30:00+00:00,144,26,12946,Starbound,88,81.0,14.99,2016-07-22,"In Starbound, you create your own story - ther...",https://cdn.akamai.steamstatic.com/steam/apps/...,"[Action, Adventure, Casual, Indie, RPG]","[Single-player, Multi-player, PvP, Online PvP,..."
29138,Splattercatgaming,Let's Play Darkwood - Episode 3 - Monitor Bliz...,2014-07-27 15:00:10+00:00,428,50,20257,Darkwood,66,80.0,14.99,2017-08-17,Darkwood provides a new perspective on surviva...,https://cdn.akamai.steamstatic.com/steam/apps/...,"[Action, Adventure, Indie, RPG]","[Single-player, Steam Achievements, Full contr..."
33715,Splattercatgaming,Reign of Kings Gameplay (Series 2) - Episode 8...,2015-04-11 17:00:04+00:00,504,47,23628,Reign Of Kings,43,NaN,9.99,2015-12-15,Struggle to survive in a harsh medieval world....,https://cdn.akamai.steamstatic.com/steam/apps/...,"[Action, Adventure, Indie, RPG, Simulation]","[Multi-player, PvP, Online PvP, LAN PvP, Co-op]"


In [87]:
df_full['Upload Date'] = pd.to_datetime(df_full['Upload Date'], errors='coerce').dt.tz_localize(None)
df_full[df_full['Release date (steam)'] > df_full['Upload Date']]

,Channel Title,Video Title,Upload Date,Likes,Comments,Views,Game Played,Video Count,Metacritic Score,Price (USD),Release date (steam),Description,header_image,Genres,Tags
0,AngryJoeShow,"AJS News - Wizards D&D OGL Controversy, Dead S...",2023-01-17 22:58:18,6763,646,135178,Dead Space,77,NaN,59.99,2023-01-27,The sci-fi survival-horror classic Dead Space ...,https://cdn.akamai.steamstatic.com/steam/apps/...,"[Action, Adventure]","[Single-player, Steam Achievements, Full contr..."
1,AngryJoeShow,"AJS News - G4TV Shuts Down, John Carpenter Dea...",2022-10-17 22:42:59,7657,813,155545,Dead Space,77,NaN,59.99,2023-01-27,The sci-fi survival-horror classic Dead Space ...,https://cdn.akamai.steamstatic.com/steam/apps/...,"[Action, Adventure]","[Single-player, Steam Achievements, Full contr..."
2,AngryJoeShow,"AJS News- Blizzard's Cosby suite, ScarJo sues ...",2021-08-03 02:03:46,11401,968,193843,Dead Space,77,NaN,59.99,2023-01-27,The sci-fi survival-horror classic Dead Space ...,https://cdn.akamai.steamstatic.com/steam/apps/...,"[Action, Adventure]","[Single-player, Steam Achievements, Full contr..."
3,AngryJoeShow,"AJS News - Ubisoft Ends Elite Squad, NEW SUPER...",2021-07-26 22:52:43,8983,824,174994,Dead Space,77,NaN,59.99,2023-01-27,The sci-fi survival-horror classic Dead Space ...,https://cdn.akamai.steamstatic.com/steam/apps/...,"[Action, Adventure]","[Single-player, Steam Achievements, Full contr..."
4,AngryJoeShow,"AJS News - Dead Space Rumors, EA Buys Mobile S...",2021-06-28 22:24:24,8760,545,162494,Dead Space,77,NaN,59.99,2023-01-27,The sci-fi survival-horror classic Dead Space ...,https://cdn.akamai.steamstatic.com/steam/apps/...,"[Action, Adventure]","[Single-player, Steam Achievements, Full contr..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45759,Splattercatgaming,Let's Play Shadowrun : Dragonfall - Episode 5 ...,2014-03-03 16:00:05,473,36,32655,Shadowrun: Dragonfall,48,87.0,14.99,2014-09-18,Shadowrun: Dragonfall - Director s Cut is a st...,https://cdn.akamai.steamstatic.com/steam/apps/...,"[Adventure, Indie, RPG, Strategy]","[Single-player, Steam Achievements, Steam Trad..."
45760,Splattercatgaming,Let's Play Shadowrun : Dragonfall - Episode 4 ...,2014-03-02 16:00:06,531,57,40216,Shadowrun: Dragonfall,48,87.0,14.99,2014-09-18,Shadowrun: Dragonfall - Director s Cut is a st...,https://cdn.akamai.steamstatic.com/steam/apps/...,"[Adventure, Indie, RPG, Strategy]","[Single-player, Steam Achievements, Steam Trad..."
45761,Splattercatgaming,Let's Play Shadowrun : Dragonfall - Episode 3 ...,2014-03-01 16:00:07,670,77,46514,Shadowrun: Dragonfall,48,87.0,14.99,2014-09-18,Shadowrun: Dragonfall - Director s Cut is a st...,https://cdn.akamai.steamstatic.com/steam/apps/...,"[Adventure, Indie, RPG, Strategy]","[Single-player, Steam Achievements, Steam Trad..."
45762,Splattercatgaming,Let's Play Shadowrun : Dragonfall - Episode 2 ...,2014-02-28 16:00:03,761,51,64944,Shadowrun: Dragonfall,48,87.0,14.99,2014-09-18,Shadowrun: Dragonfall - Director s Cut is a st...,https://cdn.akamai.steamstatic.com/steam/apps/...,"[Adventure, Indie, RPG, Strategy]","[Single-player, Steam Achievements, Steam Trad..."


In [86]:
df_full.dtypes

Channel Title                        object
Video Title                          object
Upload Date             datetime64[ns, UTC]
Likes                                 int32
Comments                              int32
Views                                 int32
Game Played                          object
Video Count                           int64
Metacritic Score                    float64
Price (USD)                         float64
Release date (steam)         datetime64[ns]
Description                          object
header_image                         object
Genres                               object
Tags                                 object
dtype: object

In [88]:
df_full['Release date (steam)'] - df_full['Upload Date']

0         9 days 01:01:42
1       101 days 01:17:01
2       541 days 21:56:14
3       549 days 01:07:17
4       577 days 01:35:36
               ...       
45842   -3 days +08:59:28
45843   -3 days +08:59:28
45844   -1 days +16:59:45
45845   -1 days +16:59:45
45846   -1 days +16:59:45
Length: 45847, dtype: timedelta64[ns]

In [90]:
from datetime import timedelta

week = timedelta(days=7)

In [97]:
df_full['At release'] = (-week <= df_full['Release date (steam)'] - df_full['Upload Date']) & (df_full['Release date (steam)'] - df_full['Upload Date'] <= week)

In [100]:
df_full.drop(columns=['Release date (steam)'], inplace=True)

In [101]:
df_full.to_pickle('./streamlit/dffull.pkl')

In [110]:
games1 = pd.DataFrame(df_full[['Game Played', 'Views']].groupby(by=['Game Played'])['Views'].sum().astype('int'))

In [119]:
games1['Videos Featuring Game'] = df_full[['Game Played', 'Views']].groupby(by=['Game Played'])['Views'].count().astype('int')
games1.sort_values(by=['Videos Featuring Game', 'Views'], ascending=False)

,Views,Videos Featuring Game
Game Played,,
Minecraft,-2147483648,2926
Factorio,45846339,2122
Europa Universalis IV,34354445,1558
Garry's Mod,-2147483648,1544
RimWorld,28199163,1209
...,...,...
Endless Legend,224896,10
Space Run Galaxy,130189,10
Magicka 2,118326,10


In [120]:
games1.describe()

,Views,Videos Featuring Game
count,612.0,612.0
mean,"25,432,785.959150326",74.91339869281046
std,"192,017,466.73312914",201.17945352513567
min,"-2,147,483,648.0",10.0
25%,"1,740,450.75",16.75
50%,"11,590,041.0",27.0
75%,"40,857,626.75",60.25
max,"710,846,957.0","2,926.0"


In [121]:
df_full.describe()

,Likes,Comments,Views,Video Count,Metacritic Score,Price (USD)
count,"45,847.0","45,847.0","45,847.0","45,847.0","30,907.0","38,125.0"
mean,"20,006.458525094335","1,734.2833336968613","1,086,737.010905839",614.2967478788144,83.5417866502734,28.27239816393443
std,"47,762.67969419692","5,438.737315695185","3,058,246.7808914967",826.356990660912,7.142336297792072,15.413437276065354
min,1.0,1.0,84.0,10.0,51.0,1.99
25%,323.0,32.0,"10,875.0",61.0,80.0,19.99
50%,"1,260.0",123.0,"53,762.0",191.0,85.0,24.99
75%,"16,258.0","1,000.0","780,815.0",862.0,88.0,38.99
max,"1,342,375.0","373,000.0","107,022,934.0","2,926.0",97.0,99.99


In [122]:
df_full.sample()

,Channel Title,Video Title,Upload Date,Likes,Comments,Views,Game Played,Video Count,Metacritic Score,Price (USD),Description,header_image,Genres,Tags,At release
2704,VanossGaming,"GTA5 Sumo Deathmatch - ""Testing"" a Broken Mod!",2022-01-01 00:00:16,64949,734,1821695,Grand Theft Auto V,907,96.0,NaN,"When a young street hustler, a retired bank ro...",https://cdn.akamai.steamstatic.com/steam/apps/...,"[Action, Adventure]","[Single-player, Multi-player, PvP, Online PvP,...",False


In [151]:
# df_full['Video Count Channel'] = 
df_full[['Channel Title', 'Game Played']].groupby(by=['Channel Title', 'Game Played'], sort = True)['Game Played'].count().sort_values()

Channel Title      Game Played                   
AngryJoeShow       A Plague Tale: Innocence             1
Splattercatgaming  Bendy and the Ink Machine            1
Many A True Nerd   Death Stranding                      1
Splattercatgaming  Beat Cop                             1
Many A True Nerd   Diaries of a Spaceport Janitor       1
                                                     ... 
quill18            RimWorld                           859
                   Civilization V                     874
                   Europa Universalis IV             1277
KatherineOfSky     Factorio                          1862
DanTDM             Minecraft                         1966
Name: Game Played, Length: 2590, dtype: int64

In [174]:
a = df_full[['Channel Title', 'Game Played']].groupby(by=['Channel Title', 'Game Played']).size().sort_values(ascending=False)

In [176]:
a.sort_index(level='Channel Title', sort_remaining=False)

Channel Title  Game Played                
AngryJoeShow   Cyberpunk 2077                 45
               Fallout 76                     24
               Battlefield 2042               20
               The Elder Scrolls V: Skyrim    19
               Apex Legends                   18
                                              ..
theRadBrad     Resident Evil                   1
               Apex Legends                    1
               The Elder Scrolls V: Skyrim     1
               Call of Duty                    1
               The Stanley Parable             1
Length: 2590, dtype: int64

In [169]:
a

Channel Title  Game Played               
AngryJoeShow   A Plague Tale: Innocence       1
               A Plague Tale: Requiem         1
               A Total War Saga: TROY         1
               ARK: Survival Evolved          3
               Alien: Isolation               5
                                             ..
theRadBrad     Watch Dogs 2                  35
               Watch Dogs: Legion            37
               Wolfenstein: The New Order    31
               Wolfenstein: Youngblood       12
               Zombie Army 4: Dead War        5
Name: Game Played, Length: 2590, dtype: int64

In [171]:
a.sort_values()

Channel Title      Game Played                   
AngryJoeShow       A Plague Tale: Innocence             1
Splattercatgaming  Bendy and the Ink Machine            1
Many A True Nerd   Death Stranding                      1
Splattercatgaming  Beat Cop                             1
Many A True Nerd   Diaries of a Spaceport Janitor       1
                                                     ... 
quill18            RimWorld                           859
                   Civilization V                     874
                   Europa Universalis IV             1277
KatherineOfSky     Factorio                          1862
DanTDM             Minecraft                         1966
Name: Game Played, Length: 2590, dtype: int64

In [177]:
df_full['Channel Title'].unique()

array(['AngryJoeShow', 'H2ODelirious', 'Markiplier', 'Mathas',
       'Many A True Nerd', 'theRadBrad', 'Smosh Games',
       'Splattercatgaming', 'DanTDM', 'quill18', 'jacksepticeye',
       'SeaNanners Gaming Channel', 'PartyElite', 'VanossGaming', 'Blitz',
       'KatherineOfSky', 'SmallAnt'], dtype=object)

In [178]:
df_vids

,Channel Title,Video Title,Upload Date,Likes,Comments,Views,Game Played
0,AngryJoeShow,Is Warhammer: Man O' War Corsair BETTER than S...,2023-01-20 17:08:39+00:00,1889,380,60972,2023
1,AngryJoeShow,"AJS News - Callisto Protocol Controversy, Blac...",2023-01-17 22:58:20+00:00,6127,602,144970,The Callisto Protocol
2,AngryJoeShow,"AJS News - Wizards D&D OGL Controversy, Dead S...",2023-01-17 22:58:18+00:00,6763,646,135178,Dead Space
4,AngryJoeShow,BattleBit Remastered is BETTER than Battlefiel...,2023-01-14 23:42:19+00:00,5327,1200,131807,Battlefield 2042
8,AngryJoeShow,The Game Awards 2022 - FULL LENGTH ANGRY REACT...,2023-01-03 00:01:18+00:00,2619,867,97435,League of Legends
...,...,...,...,...,...,...,...
78369,VanossGaming,"Minecraft Funny Moments - Minecart Fun, Scarin...",2013-05-25 21:30:37+00:00,118264,2700,7872364,Minecraft
78372,VanossGaming,"Black Ops 2 Funny Random Moments - Hot Dog, Mi...",2013-05-16 00:39:27+00:00,88085,1400,5839700,Call of Duty: Black Ops II
78373,VanossGaming,Black Ops 2 Funny Kills & Killcams - Trophy Sy...,2013-05-14 00:18:07+00:00,49981,327,3481279,Call of Duty: Black Ops II
78379,VanossGaming,"MW3 Funny Moments - Retard Run Kills, Angry Ki...",2013-02-23 01:03:17+00:00,49587,1400,3442404,Call of Duty: Black Ops II
